In [4]:
from datasets import load_dataset
import polars as pl
from IPython.display import display, Markdown, Latex
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import rich
from rich.syntax import Syntax
from rich.console import Console

In [2]:
cc_dataset = load_dataset("deepmind/code_contests")["train"]

In [34]:
console = Console()

first_n_samples = {}

def print_code_snippet(snippet: str, console: Console):
    formatted_snippet = Syntax(
        snippet,
        "python",
        theme="monokai",
        line_numbers=True,
    )
    console.print(formatted_snippet)

for i, sample in enumerate(cc_dataset):
    for k, v in sample.items(): # type: ignore
        if k not in first_n_samples:
            first_n_samples[k] = []
        first_n_samples[k].append(v)
    for solution in sample["solutions"]['solution']:
        print_code_snippet(solution, console)
    test_inputs = sample["public_tests"]['input']  + sample["private_tests"]['input']
    test_outputs = sample["public_tests"]['output'] + sample["private_tests"]['output']
    print(test_inputs)
    print(test_outputs)
    if i > 1:
        break

  1 for _ in range(input()):                                                                                       
  2     try:                                                                                                       
  3         eval(raw_input())                                                                                      
  4         print 'YES'                                                                                            
  5     except TypeError:                                                                                          
  6         print 'YES'                                                                                            
  7     except:                                                                                                    
  8         print 'NO'                                                                                             

   1 for _ in range(input()):                                                                                      
   2     ins = raw_input().strip()                                                                                 
   3     stck = []                                                                                                 
   4     res = "YES"                                                                                               
   5     for x in ins:                                                                                             
   6         if x == "(":                                                                                          
   7             stck.append(x)                                                                                    
   8         else:                                                                                                 
   9             if len(stck)>0:                                                                                   
  10                 stck.pop()                                                                                    
  11             else:                                                                                             
  12                 res = "NO"                                                                                    
  13                 break                                                                                         
  14     if len(stck) > 0: res = "NO"                                                                              
  15     print res                                                                                                 

  1 for _ in range(input()):                                                                                       
  2     try: eval(raw_input()); print 'YES'                                                                        
  3     except TypeError: print 'YES'                                                                              
  4     except: print 'NO'                                                                                         

['3\n((()))\n(())()\n()(()']
['YES\nYES\nNO']


   1 import math                                                                                                   
   2 no_of_testcases = int(input())                                                                                
   3 for each in range(no_of_testcases):                                                                           
   4     dist = int(input())                                                                                       
   5     point_1 = map(int,raw_input().split())                                                                    
   6     point_2 = map(int,raw_input().split())                                                                    
   7     point_3 = map(int,raw_input().split())                                                                    
   8     point_12 =math.sqrt( math.pow((point_1[0] -point_2[0]),2) + math.pow((point_1[1] -point_2[1]),2))         
   9     point_23 =math.sqrt( math.pow((point_2[0] -point_3[0]),2) + math.pow((point_2[1] -point_3[1]),2))         
  10     point_31 =math.sqrt( math.pow((point_3[0] -point_1[0]),2) + math.pow((point_3[1] -point_1[1]),2))         
  11     count =0                                                                                                  
  12     if point_12 <= dist:                                                                                      
  13         count =count+1                                                                                        
  14     if point_23 <= dist:                                                                                      
  15         count =count+1                                                                                        
  16     if point_31 <= dist:                                                                                      
  17         count =count+1                                                                                        
  18     if count >=2:                                                                                             
  19         print "yes"                                                                                           
  20     else:                                                                                                     
  21         print [3

...

   1 # cook your code here                                                                                         
   2                                                                                                               
   3 def getModuloOf(a, two):                                                                                      
   4     i=1;                                                                                                      
   5     b = int(two[:i]);                                                                                         
   6     while(b<a and i<len(two)):                                                                                
   7         i=i+1;                                                                                                
   8         b=int(two[:i]);                                                                                       
   9     if(b<a or i==len(two)):                                                                                   
  10         return b%a;                                                                                           
  11     else:                                                                                                     
  12         rem=b%a;                                                                                              
  13         s=str(rem)+two[i:];                                                                                   
  14         return getModuloOf(a,s);                                                                              
  15                                                                                                               
  16 def findHCF(a, b):                                                                                            
  17     if(a==0):                                                                                                 
  18         return b;                                                                                             
  19     return findHCF(b%a,a);                                                                                    
  20                                                                                                               
  21 t=int(raw_input());                                                                                           
  22 #waste=raw_input();                                                                                           
  23 while(t>0):                                                                                                   
  24     a,two = raw_input().split(' ');                                                                           
  25     a=int(a);                                                                                                 
  26     if(a==0):                                                                                                 
  27         print two;                                                                                            
  28     else:                                                                                                     
  29         b=getModuloOf(a,two);                                                                                 
  30         ans=findHCF(b,a);                                                                                     
  31         print ans;                                                                                            
  32     t=t-1;                                                                                                

   1 def gcd(a,b):                                                                                                 
   2    if b==0:                                                                                                   
   3       return a                                                                                                
   4    else:                                                                                                      
   5       return gcd(b,a%b)                                                                                       
   6                                                                                                               
   7 t=input()                                                                                                     
   8 while t>0:                                                                                                    
   9    a,b = map(int, raw_input().split(" "))                                                                     
  10    print gcd(a,b)                                                                                             
  11    t-=1                                                                                                       

  1 def gcd(a, b):                                                                                                 
  2     if(a == 0): return b;                                                                                      
  3     return gcd(b % a, a);                                                                                      
  4 t = input();                                                                                                   
  5 while(t > 0):                                                                                                  
  6     a, b = map(int, raw_input().split());                                                                      
  7     print gcd(a, b);                                                                                           
  8     t -= 1;                                                                                                    

   1 import math                                                                                                   
   2 import sys                                                                                                    
   3 def parseIntList(str):                                                                                        
   4  return [long(x) for x in str.split()]                                                                        
   5 def printBS(li):                                                                                              
   6  if len(li) is 0:                                                                                             
   7   print                                                                                                       
   8  else:                                                                                                        
   9   for i in range(len(li)-1):                                                                                  
  10    print li[i],                                                                                               
  11   print li[-1]                                                                                                
  12 def gcd(a,b):                                                                                                 
  13  if b==0: return a                                                                                            
  14  return gcd(b,a%b)                                                                                            
  15 cases=input()                                                                                                 
  16 for case in range(cases):                                                                                     
  17  b,a=raw_input().split()                                                                                      
  18  b=int(b)                                                                                                     
  19  if b==0:                                                                                                     
  20   print a                                                                                                     
  21  else:                                                                                                        
  22   num=0                                                                                                       
  23   for i in a:                                                                                                 
  24    num=(num*10+int(i))%b                                                                                      
  25   print gcd(b,num)                                                                                            

['2\n2 6\n10 11']
['2\n1\n']
